# ビデオポートの認識確認

In [ ]:
!ls -ltrh /dev/video*

# USBカメラに接続

In [ ]:
from jetcam.usb_camera import USBCamera

#TODO change capture_device if incorrect for your system
camera = USBCamera(width=320, height=240, capture_width=640, capture_height=480, capture_device=0)

In [ ]:
image = camera.read()

# USBカメラの画像を表示

In [ ]:
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

image_widget = ipywidgets.Image(format='jpeg')

image_widget.value = bgr8_to_jpeg(image)

display(image_widget)

# カメラ画像の更新処理

In [ ]:
camera.running = True

def update_image(change):
    global image
    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)
    
camera.observe(update_image, names='value')

# 顔認識用の画像登録処理

In [ ]:
import face_recognition

def register_face():
    global image, my_face_encoding
    
    # カメラ画像を取得しmyfacde.jpgという名前で保存
    face_image = Image.frombytes('RGB', (320,240), image)
    face_image.save("myface.jpg")
    register_face_widget.value = bgr8_to_jpeg(image)
    
    # 保存した写真を次の顔認識処理に備えて読み込み　
    picture_of_me = face_recognition.load_image_file("myface.jpg")
    my_face_encoding = face_recognition.face_encodings(picture_of_me)[0]

# 顔認識用の画像登録ボタンの作成

In [ ]:
from PIL import Image

register_button = ipywidgets.Button(description='Register face')
register_button.on_click(lambda c: register_face())

with open("ready_img.jpg", "rb") as file:
    default_image = file.read()
register_face_widget = ipywidgets.Image(value=default_image, width=camera.width, height=camera.height)

# 顔認証処理

In [ ]:
def recognition_face():
    global image, my_face_encoding
    
    # カメラ画像の取り込み
    unknown_face_encoding = face_recognition.face_encodings(image)[0]

    # 登録画像とカメラ画像で比較し結果を取得
    results = face_recognition.compare_faces([my_face_encoding], unknown_face_encoding)

    # 結果をsuccess, failedでWidgetに表示
    if results[0] == True:
        result_widget.value = "success"
    else:
        result_widget.value = "failed"

# 顔認識ボタンの作成

In [ ]:
recognision_button = ipywidgets.Button(description='Recognision face')
recognision_button.on_click(lambda c: recognition_face())

# 結果表示Widgetの作成

In [ ]:
result_widget = ipywidgets.Text(description='result', value='')

# Widgetを画面に表示

In [ ]:
tool_widget = ipywidgets.VBox([
    ipywidgets.HBox([
        register_face_widget,
        register_button,
    ]),
    ipywidgets.HBox([
        image_widget,
        recognision_button
    ]),
    result_widget
])

In [ ]:
display(tool_widget)

# カメラ等の終了処理

In [ ]:
import os
os._exit(00)